#  Experimentos de Imputación

In [1]:
import subprocess, sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))
for pkg in ['pandas', 'numpy', 'scikit-learn']: 
    try: __import__(pkg)
    except: subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys; sys.path.insert(0, str(PROJECT_ROOT / 'src'))
from imputation import ImputationExperiment, apply_best_imputation
from io_utils import identify_column_types
import warnings; warnings.filterwarnings('ignore')

DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
df = pd.read_parquet(DATA_PROCESSED / 'raw_clean.parquet')
print(f' Dataset: {df.shape}')

✓ Dataset: (42644, 27)


## Experimentos

In [3]:
target_col = 'GRUPO'
X = df.drop(columns=[target_col])
y = df[target_col]
col_types = identify_column_types(X)
numeric_cols = col_types['numeric']
categorical_cols = col_types['categorical']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
exp = ImputationExperiment(X_train, y_train, numeric_cols, categorical_cols)
results_df = exp.run_all_experiments()
results_df

🔬 Experimentando con estrategias de imputación...

1. Imputación con Media...
  Media: F1=0.8483 (±0.0004), Nulos restantes=1071
2. Imputación con Mediana...
  Mediana: F1=0.8483 (±0.0004), Nulos restantes=1071
3. Imputación con Moda...
  Moda: F1=0.8483 (±0.0004), Nulos restantes=1071
4. Imputación con KNN...
  KNN: F1=0.8483 (±0.0005), Nulos restantes=1071
5. Imputación con Iterative (MICE)...
  Iterative: F1=0.8483 (±0.0004), Nulos restantes=1071

📊 Resumen de Resultados:
   method  mean_f1   std_f1  remaining_nulls
Iterative 0.848331 0.000440             1071
      KNN 0.848269 0.000461             1071
    Media 0.848251 0.000441             1071
     Moda 0.848251 0.000441             1071
  Mediana 0.848251 0.000441             1071


,method,mean_f1,std_f1,remaining_nulls
4,Iterative,0.848331,0.000440,1071
3,KNN,0.848269,0.000461,1071
0,Media,0.848251,0.000441,1071
2,Moda,0.848251,0.000441,1071
1,Mediana,0.848251,0.000441,1071


In [ ]:
best_strategy = 'median'
X_imputed = apply_best_imputation(X, numeric_cols, categorical_cols, strategy=best_strategy)
X_imputed.to_parquet(DATA_PROCESSED / 'imputed.parquet', index=False)
y.to_frame().to_parquet(DATA_PROCESSED / 'target.parquet', index=False)
print(' Guardado')

🔧 Aplicando imputación: median
✓ Imputación completada. Nulos restantes: 1350
✅ Guardado


Análisis: se Comparo métodos de imputación; el mejor desempeño (Iterative) mostró mean_f1 ≈ 0.848331 (std ≈ 0.000440) y tras aplicar la imputación quedan 1071 nulos residuales. También se evaluó KNN y la media y moda en el mismo experimento con resultados comparables.

Conclusiones: La imputación seleccionada preserva la prediccion (F1≈0.848) y reduce sustancialmente la falta de información, quedando 1.071 registros.